<a href="https://colab.research.google.com/github/pliniosa/PySpark/blob/main/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PySpark hands on 1
O Objetivo desse material e estudar o framework spark

## Instalando pacotes necessários

In [ ]:
from IPython.display import clear_output

!pip install --upgrade pip #atualiza o gestor de pacotes do Python
!pip install findspark
!pip install pyspark #instalar a lib pyspark (lib = framework)

clear_output(wait=True) #limpar a saída

In [ ]:
import findspark, pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [ ]:
findspark.init()
spark = SparkSession.builder.getOrCreate()

##Lendo Fonte de dados do GItHub

In [ ]:
url = 'https://raw.githubusercontent.com/edsonlourenco/public_datasets/main/Carros.csv'

spark.sparkContext.addFile(url) # Faz o download da base de dados
csv_cars = SparkFiles.get('Carros.csv') #path do arquivo na tmp do arquivo
df_cars = spark.read.csv(csv_cars, header=True, inferSchema=True, sep=';')

##Checando schema


In [ ]:
df_cars.printSchema()

root
 |-- Consumo: integer (nullable = true)
 |-- Cilindros: integer (nullable = true)
 |-- Cilindradas: integer (nullable = true)
 |-- RelEixoTraseiro: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Tempo: integer (nullable = true)
 |-- TipoMotor: integer (nullable = true)
 |-- Transmissao: integer (nullable = true)
 |-- Marchas: integer (nullable = true)
 |-- Carburadors: integer (nullable = true)
 |-- HP: integer (nullable = true)



##Contar os registros


In [ ]:
df_cars.count() #total de rows (linha)

32

## Exibindo os Dados

In [ ]:
df_cars.show(truncate=False)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors|HP |
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|21     |6        |160        |39             |262 |1646 |0        |1          |4      |4          |110|
|21     |6        |160        |39             |2875|1702 |0        |1          |4      |4          |110|
|228    |4        |108        |385            |232 |1861 |1        |1          |4      |1          |93 |
|214    |6        |258        |308            |3215|1944 |1        |0          |3      |1          |110|
|187    |8        |360        |315            |344 |1702 |0        |0          |3      |2          |175|
|181    |6        |225        |276            |346 |2022 |1        |0          |3      |1          |105|
|143    |8        |360        |321            |357 |158

## Selecionando colunas

In [ ]:
df_cars.select('Tempo').show(truncate=False) # SELECT Tempo FROM tb_cars

+-----+
|Tempo|
+-----+
|1646 |
|1702 |
|1861 |
|1944 |
|1702 |
|2022 |
|1584 |
|20   |
|229  |
|183  |
|189  |
|174  |
|176  |
|18   |
|1798 |
|1782 |
|1742 |
|1947 |
|1852 |
|199  |
+-----+
only showing top 20 rows



## Selecionando multiplas colunas

In [ ]:
cols = ['Tempo', 'Cilindros', 'RelEixoTraseiro']
df_cars.select(cols).show(truncate=False)

+-----+---------+---------------+
|Tempo|Cilindros|RelEixoTraseiro|
+-----+---------+---------------+
|1646 |6        |39             |
|1702 |6        |39             |
|1861 |4        |385            |
|1944 |6        |308            |
|1702 |8        |315            |
|2022 |6        |276            |
|1584 |8        |321            |
|20   |4        |369            |
|229  |4        |392            |
|183  |6        |392            |
|189  |6        |392            |
|174  |8        |307            |
|176  |8        |307            |
|18   |8        |307            |
|1798 |8        |293            |
|1782 |8        |3              |
|1742 |8        |323            |
|1947 |4        |408            |
|1852 |4        |493            |
|199  |4        |422            |
+-----+---------+---------------+
only showing top 20 rows



## Melhorando a identação do codigo

In [ ]:
cols = ['Tempo', 'Cilindros', 'RelEixoTraseiro']
(
    df_cars
        .orderBy('Tempo')
        .select(cols)
        .show(truncate=False)
)

+-----+---------+---------------+
|Tempo|Cilindros|RelEixoTraseiro|
+-----+---------+---------------+
|18   |8        |307            |
|20   |4        |369            |
|145  |8        |422            |
|146  |8        |354            |
|155  |6        |362            |
|167  |4        |443            |
|169  |4        |377            |
|173  |8        |315            |
|174  |8        |307            |
|176  |8        |307            |
|183  |6        |392            |
|186  |4        |411            |
|189  |6        |392            |
|189  |4        |408            |
|199  |4        |422            |
|229  |4        |392            |
|1541 |8        |373            |
|1584 |8        |321            |
|1646 |6        |39             |
|1687 |8        |276            |
+-----+---------+---------------+
only showing top 20 rows



## Agrupamento

In [ ]:
from pyspark.sql import functions as F

(
    df_cars
      .groupBy('Cilindros')
      .agg(
          F.count('*').alias('Quantidade')
          ).show()
)

+---------+----------+
|Cilindros|Quantidade|
+---------+----------+
|        6|         7|
|        4|        11|
|        8|        14|
+---------+----------+



In [ ]:
(
    df_cars
            .groupBy('TipoMotor')
            .agg(F.sum('Peso').alias('Peso_Total'))
            .orderBy('TipoMotor')
            .select(
                F.lit('Carros')
                .alias('Categorias'), 
                F.col('TipoMotor'), 
                F.col('Peso_Total')
            ).show()
)

+----------+---------+----------+
|Categorias|TipoMotor|Peso_Total|
+----------+---------+----------+
|    Carros|        0|     25471|
|    Carros|        1|     14778|
+----------+---------+----------+



## Extraindo a média (avg - average)

In [ ]:
(
    df_cars
            .groupBy('TipoMotor')
            .agg(F.avg('Cilindradas').alias('Cilindradas_Med'))
            .orderBy('TipoMotor')
            .select(
                F.lit('Carros').alias('Categorias'), 
                F.col('TipoMotor'), 
                F.col('Cilindradas_Med')
            ).show()
)

+----------+---------+-----------------+
|Categorias|TipoMotor|  Cilindradas_Med|
+----------+---------+-----------------+
|    Carros|        0|            781.0|
|    Carros|        1|816.0714285714286|
+----------+---------+-----------------+



## Conversão de tipos

In [ ]:
from pyspark.sql.types import IntegerType
(
    df_cars
            .groupBy('TipoMotor')
            .agg(F.avg('Cilindradas').cast(IntegerType()).alias('Cilindradas_Med'))
            .select(
                F.lit('Carros').alias('Categorias'), 
                F.col('TipoMotor'), 
                F.col('Cilindradas_Med')
            ).show()
)

+----------+---------+---------------+
|Categorias|TipoMotor|Cilindradas_Med|
+----------+---------+---------------+
|    Carros|        1|            816|
|    Carros|        0|            781|
+----------+---------+---------------+



## Filtrando os dados

In [ ]:
from pyspark.sql import functions as F

#df_cars.filter(F.col('TipoMotor') == 1).show(truncate=False)
#df_cars.where(F.col('TipoMotor') == 1).show(truncate=False)
df_cars.where(df_cars.TipoMotor == 1).show(truncate=False)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors|HP |
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|228    |4        |108        |385            |232 |1861 |1        |1          |4      |1          |93 |
|214    |6        |258        |308            |3215|1944 |1        |0          |3      |1          |110|
|181    |6        |225        |276            |346 |2022 |1        |0          |3      |1          |105|
|244    |4        |1467       |369            |319 |20   |1        |0          |4      |2          |62 |
|228    |4        |1408       |392            |315 |229  |1        |0          |4      |2          |95 |
|192    |6        |1676       |392            |344 |183  |1        |0          |4      |4          |123|
|178    |6        |1676       |392            |344 |189